In [98]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import metrics

In [99]:
data_xls = pd.read_excel('辽宁钢铁行业月用电量.xls')

In [100]:
data_xls

,月份,辽宁钢铁行业月用电量（亿千万时）,钢铁产量,汽车产量,水泥产量,金属切削机床,大中型拖拉机,鞍钢股价,本钢股价,凌钢股价
0,2007M01,13.856687,354.62,78.72,8370.22,4.12,14440.0,12.95,6.50,6.10
1,2007M02,11.941298,318.87,59.65,6500.82,3.35,13941.0,12.87,8.09,7.58
2,2007M03,14.555307,368.91,88.96,9308.57,4.78,26231.0,14.57,8.55,8.28
3,2007M04,13.656541,366.22,83.98,11643.56,5.41,22263.0,17.64,13.35,11.03
4,2007M05,13.143799,374.81,74.35,12869.06,5.34,14824.0,18.34,13.89,10.44
...,...,...,...,...,...,...,...,...,...,...
153,2019M10,19.591677,609.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,2019M11,21.042283,630.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,2019M12,21.137609,648.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,2020M01,22.502425,621.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
# data_xls.drop(['月份'，], axis=1, inplace=True)
# data_xls

In [105]:
data_origin = np.array(data_xls)[:84,1:6]
print(data_origin)

[[13.85668698 354.62 78.72 8370.22 4.12]
 [11.94129775 318.87 59.65 6500.82 3.35]
 [14.5553069 368.91 88.96 9308.57 4.78]
 [13.65654085 366.22 83.98 11643.56 5.41]
 [13.14379943 374.81 74.35 12869.06 5.34]
 [13.89728023 407.8 75.92 12733.27 5.51]
 [13.82012254 371.56 67.88 11748.24 5.27]
 [13.79591637 347.4 66.14 12081.19 5.31]
 [13.98148773 371.86 78.3 12520.21 5.73]
 [13.27588781 358.27 75.17 12379.76 5.3]
 [13.96807882 345.8 86.52 12547.31 5.24]
 [14.28233571 369.6 84.87 12328.41 5.06]
 [14.25003836 336.4 84.3 8268.01 5.07]
 [12.63311045 335.34 67.21 5958.28 4.69]
 [13.46243023 366.4 106.57 11322.94 5.85]
 [12.66591199 343.26 100.41 12488.42 6.03]
 [15.74872233 399.75 89.49 13198.24 7.13]
 [13.96355619 409.85 86.42 13181.24 5.89]
 [14.0412472 380.07 75.72 11943.23 5.4]
 [14.73483898 372.28 62.58 11985.87 5.31]
 [14.43095576 355.84 79.18 12422.85 5.11]
 [13.07001702 323.2 73.03 12309.27 5.0]
 [11.2565999 307.96 71.35 12679.63 4.35]
 [12.75663418 342.5 68.57 12527.17 3.81]
 [15.180401

In [143]:
data_y_origin = data_origin[:,1]
data_y_origin

array([354.62, 318.87, 368.91, 366.22, 374.81, 407.8, 371.56, 347.4,
       371.86, 358.27, 345.8, 369.6, 336.4, 335.34, 366.4, 343.26, 399.75,
       409.85, 380.07, 372.28, 355.84, 323.2, 307.96, 342.5, 353.16,
       330.71, 378.61, 348.83, 439.96, 428.16, 456.05, 469.65, 436.27,
       435.96, 422.55, 462.8, 446.2, 401.6, 463.3, 480.0, 480.1, 489.7,
       468.1, 476.3, 487.6, 477.7, 487.6, 488.4, 448.3, 420.9, 493.5,
       460.7, 524.2, 532.1, 508.5, 508.1, 485.7, 473.5, 440.0, 435.4,
       476.3, 400.8, 507.1, 495.9, 511.4, 588.5, 521.6, 457.8, 445.6,
       455.2, 501.2, 543.1, 531.1, 509.5, 570.3, 523.1, 562.2, 583.0,
       569.8, 558.1, 578.9, 572.4, 595.95, 594.8], dtype=object)

In [109]:
data_x = np.delete(data_origin,1, axis = 1)
print(data_x)

[[13.85668698 78.72 8370.22 4.12]
 [11.94129775 59.65 6500.82 3.35]
 [14.5553069 88.96 9308.57 4.78]
 [13.65654085 83.98 11643.56 5.41]
 [13.14379943 74.35 12869.06 5.34]
 [13.89728023 75.92 12733.27 5.51]
 [13.82012254 67.88 11748.24 5.27]
 [13.79591637 66.14 12081.19 5.31]
 [13.98148773 78.3 12520.21 5.73]
 [13.27588781 75.17 12379.76 5.3]
 [13.96807882 86.52 12547.31 5.24]
 [14.28233571 84.87 12328.41 5.06]
 [14.25003836 84.3 8268.01 5.07]
 [12.63311045 67.21 5958.28 4.69]
 [13.46243023 106.57 11322.94 5.85]
 [12.66591199 100.41 12488.42 6.03]
 [15.74872233 89.49 13198.24 7.13]
 [13.96355619 86.42 13181.24 5.89]
 [14.0412472 75.72 11943.23 5.4]
 [14.73483898 62.58 11985.87 5.31]
 [14.43095576 79.18 12422.85 5.11]
 [13.07001702 73.03 12309.27 5.0]
 [11.2565999 71.35 12679.63 4.35]
 [12.75663418 68.57 12527.17 3.81]
 [15.18040114 67.37 7580.12 3.41]
 [13.37540347 85.28 8289.98 4.08]
 [14.73297642 116.9 12206.74 5.41]
 [15.78046701 118.12 14677.55 5.53]
 [17.72041254 114.76 14888.43 5.

In [111]:
# print(data_origin)

In [113]:
# #Max-Min标准化
# #建立MinMaxScaler对象
scaler = preprocessing.MinMaxScaler()

# # 标准化处理
# scaler = preprocessing.StandardScaler()

data_x = scaler.fit_transform(data_x)
data_y = scaler.fit_transform(data_y.reshape(-1, 1))

In [6]:
# n = 0
# for i in range(data.shape[0]):
#     if data[i,0]<0:
#         n=n+1
# print(n)

In [114]:
print(data_x.shape)
print(data_y.shape)

(84, 4)
(84, 1)


In [115]:
# data.shape

In [116]:
# data_x = data[:,0]

In [117]:
# data_y = data[:,1]

In [118]:
# data_y = data[-146:,:]
# print(data_y.shape)

In [119]:
# data_x = []
# for r in range(146):
#     for c in range(12):
#         item = []
#         item.append(data[r+c, 0])
#         data_x.append(item)
# data_x = np.array(data_x)
# print(data_x)
# print(data_x.shape)

In [120]:
# data_x=data_x.reshape(84,1,1)

In [121]:
# data_x.shape

In [122]:
# data_x

In [123]:
# data_y = data_y.reshape(84,1,1)

In [124]:
data_x_train = data_x[:70,:]
data_y_train = data_y[:70,:]

data_x_test = data_x[-14:,:]
data_y_test = data_y[-14:,:]

In [125]:
print(data_x_train.shape)
print(data_y_train.shape)
print(data_x_test.shape)
print(data_y_test.shape)

(70, 4)
(70, 1)
(14, 4)
(14, 1)


In [126]:
from sklearn.model_selection import train_test_split

In [127]:
X_train, X_test, Y_train, Y_test = train_test_split(data_x_train, data_y_train, test_size=0.2, random_state=520)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(56, 4)
(56, 1)
(14, 4)
(14, 1)


In [18]:

#     model = Sequential()
 
#     model.add(LSTM(input_dim=10,output_dim=128,return_sequences=True))
#     model.add(Dropout(0.1))
 
#     model.add(LSTM(128,return_sequences=False))
#     model.add(Dropout(0.1))
 
#     model.add(Dense(output_dim=1))
#     model.add(Activation("linear"))
 
#     model.compile(loss="mse", optimizer="rmsprop")

In [19]:
# model = Sequential()

# model.add(LSTM(output_dim=128, 
#                 input_shape=(10, 1),
#                 activation='relu',
#                 return_sequences=True))
# for i in range(6):
#     model.add(LSTM(output_dim=128,
#                     activation='tanh',
#                     return_sequences=True))
# #     model.add(Dropout(0.1))

# model.add(LSTM(output_dim=1,
#                activation='linear',
#                return_sequences=False))

# model.compile(loss='mse', optimizer='adam', metrics=[metrics.mse])
# model.summary()

In [20]:
# def create_model(output_dim1, num_layer, output_dim2):
#     model = Sequential()

#     model.add(LSTM(output_dim=output_dim1, 
#                 input_shape=(1, 1),
#                 activation='relu',
#                 return_sequences=True))
#     for i in range(num_layer):
#         model.add(LSTM(output_dim=output_dim2,
#                         activation='tanh',
#                         return_sequences=True))
# #     model.add(Dropout(0.1))

#     model.add(LSTM(output_dim=1,
#                activation='tanh',
#                 return_sequences=False))

#     model.compile(loss='mse', optimizer='adam', metrics=[metrics.mse])
# #     model.summary()
#     return model

In [21]:
# from sklearn.model_selection import RandomizedSearchCV
# from skopt.space import Real, Categorical, Integer
# from keras.wrappers.scikit_learn import KerasRegressor
# # from sklearn.model_selection import train_test_split

In [22]:
# #采用暴力搜索，所有参数进行组合，然后选择最好的参数
# tuned_parameters=[{'output_dim1':Integer(28, 128),
#                    'num_layer':Integer(3,12),
#                    'output_dim2':Integer(28, 128)}]
# # 'output_dim1':Integer(28, 128),
# #             'num_layer':Integer(3,12),
# #             'output_dim2':Integer(28, 128)

In [23]:
# train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, train_size=0.80, random_state=0)

In [24]:
# model=KerasRegressor(build_fn=create_model)

In [25]:
# #采用随机搜索，给参数一个范围，然后系统随机选择参数，进行检验，然后选择最好的
# tuned_parameters={'output_dim1':[28, 64, 96,128],
#                    'num_layer':[4,5,6,7,8],
#                    'output_dim2':[28,64, 96,128]}

# lstm=RandomizedSearchCV(model,tuned_parameters,scoring='neg_mean_squared_error',n_iter=10)
 
# lstm.fit(data_x_train,data_y_train)

In [26]:
# print('最优训练器:',lstm.best_estimator_)
# print('最优训练器的精度:',lstm.best_score_)

In [27]:
# best_estimator_ = lstm.best_estimator_

In [28]:
# best_estimator_.predict(data_x_test,batch_size = 1)

In [29]:
# import sklearn
# sorted(sklearn.metrics.SCORERS.keys())

In [30]:
# from sklearn.model_selection import train_test_split
# from skopt import BayesSearchCV
# from skopt.space import Real, Categorical, Integer

In [31]:
# class FixedBayesSearchCV(BayesSearchCV):
#     """
#     Dirty hack to avoid compatibility issues with sklearn 0.2 and skopt.
#     Credit: https://www.kaggle.com/c/home-credit-default-risk/discussion/64004
#     For context, on why the workaround see:
#         - https://github.com/scikit-optimize/scikit-optimize/issues/718
#         - https://github.com/scikit-optimize/scikit-optimize/issues/762
#     """
#     def __init__(self, estimator, search_spaces, optimizer_kwargs=None,
#                 n_iter=50, scoring=None, fit_params=None, n_jobs=1,
#                 n_points=1, iid=True, refit=True, cv=None, verbose=0,
#                 pre_dispatch='2*n_jobs', random_state=None,
#                 error_score='raise', return_train_score=False):
#         """
#         See: https://github.com/scikit-optimize/scikit-optimize/issues/762#issuecomment-493689266
#         """
 
#         # Bug fix: Added this line
#         self.fit_params = fit_params
 
#         self.search_spaces = search_spaces
#         self.n_iter = n_iter
#         self.n_points = n_points
#         self.random_state = random_state
#         self.optimizer_kwargs = optimizer_kwargs
#         self._check_search_space(self.search_spaces)
 
#         # Removed the passing of fit_params to the parent class.
#         super(BayesSearchCV, self).__init__(
#                 estimator=estimator, scoring=scoring, n_jobs=n_jobs, iid=iid,
#                 refit=refit, cv=cv, verbose=verbose, pre_dispatch=pre_dispatch,
#                 error_score=error_score, return_train_score=return_train_score)
 
#     def _run_search(self, x):
#         raise BaseException('Use newer skopt')

In [32]:
# def lstm_auto_para_tuning_bayesian(model,data_x,data_y):
#     train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, train_size=0.80, random_state=0)
#     # cv：交叉验证 n_points：并行采样的超参组数
# #     model=KerasRegressor(build_fn=create_model)
#     opt = FixedBayesSearchCV(model,cv=5,n_points=2,n_jobs=4,verbose=1,
#         search_spaces={
#             'output_dim1':Integer(12, 48),
#             'num_layer':Integer(3,12),
#             'output_dim2':Integer(12, 48)
#         },
#          fit_params={
#                  'eval_set':[(test_x[:,0,0], test_y[:,0,0])],
#                  'eval_metric': 'mse',
#                  'early_stopping_rounds': 50
#                  })
#     opt.fit(train_x,train_y)
#     print("val. score: %s" % opt.best_score_)
#     print("test score: %s" % opt.score(test_x, test_y))
#     print("Best parameters: ", opt.best_params_)
#     print("Best estimator:", opt.best_estimator_)

In [33]:
# model=KerasRegressor(build_fn=create_model)

In [34]:
# lstm_auto_para_tuning_bayesian(model,data_x,data_y)

In [35]:
# model = create_model(24, 4, 24)

Instructions for updating:
Colocations handled automatically by placer.


e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(1, 1), activation="relu", return_sequences=True, units=24)`
  import sys
e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, units=24)`
  # This is added back by InteractiveShellApp.init_path()
e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=False, units=1)`
  app.launch_new_instance()


In [36]:
# lstm_auto_para_tuning_bayesian(model,data_x,data_y)

In [154]:
model = Sequential()
model.add(Dense(4, activation='relu',input_shape=(X_train.shape[1],)))
model.add(Dense(6, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer = 'sgd',# 还可以通过optimizer = optimizers.RMSprop(lr=0.001)来为优化器指定参数.optimizer='rmsprop'
              loss='mean_squared_error', # 等价于loss = losses.binary_crossentropy
              metrics=['accuracy']) # 等价于metrics = [metircs.binary_accuracy]
model.fit(X_train, Y_train,epochs=80, batch_size=20, verbose=0, validation_data=(X_test,Y_test))

In [155]:
model.evaluate(X_test,Y_test)

14/14 [==============================] - 0s 71us/step


[0.07816202193498611, 0.0714285746216774]

In [76]:
# model.fit(data_x_train, data_y_train, epochs=20, batch_size=20, verbose=1)

In [156]:
# 评估模型,不输出预测结果
loss,accuracy = model.evaluate(X_test,Y_test)
print('\ntest loss','{:.5f}'.format(loss))
print('accuracy','{:.5f}'.format(accuracy))

14/14 [==============================] - 0s 72us/step

test loss 0.07816
accuracy 0.07143


In [157]:
y_pred = model.predict(data_x_test,batch_size = 1)

In [158]:
y_pred

array([[0.5394414 ],
       [0.54390645],
       [0.5470811 ],
       [0.5117288 ],
       [0.55145466],
       [0.540781  ],
       [0.5423729 ],
       [0.5359737 ],
       [0.5228757 ],
       [0.52800155],
       [0.5393841 ],
       [0.53545654],
       [0.544151  ],
       [0.5533332 ]], dtype=float32)

In [41]:
# data_y_test[-10:,0]

In [42]:
# data_x_test[-10:,0,-1]

In [43]:
# t = []
# for i in range(2000):
#     if data_y_test[:,0][i]>data_x_test[:,0,-1][i]:
#         t.append(1)
#     else:
#         t.append(0)
# len(t)

In [44]:
# p = []
# for i in range(2000):
#     if y_pred[:,0][i]>data_x_test[:,0,-1][i]:
#         p.append(1)
#     else:
#         p.append(0)
# len(p)

In [45]:
# n = 0
# for i in range(2000):
#     if t[i]==p[i]:
#         n=n+1
# n/2000

In [46]:
# data.shape

In [144]:
data_y_max = np.max(data_y_origin)

In [145]:
data_y_min = np.min(data_y_origin)

In [146]:
data_y_max

595.95

In [147]:
data_y_min 

307.96

In [148]:
y_test_origin = data_origin[-14:,1]

In [149]:
y_test_origin

array([501.2, 543.1, 531.1, 509.5, 570.3, 523.1, 562.2, 583.0, 569.8,
       558.1, 578.9, 572.4, 595.95, 594.8], dtype=object)

In [150]:
y_pred_to_origin = []
for i in y_pred:
    y = i*(data_y_max - data_y_min) + data_y_min
    y_pred_to_origin.append(y)

In [151]:
y_pred_to_origin

[array([474.10693], dtype=float32),
 array([470.93274], dtype=float32),
 array([464.5332], dtype=float32),
 array([426.76337], dtype=float32),
 array([474.70056], dtype=float32),
 array([480.66336], dtype=float32),
 array([481.17557], dtype=float32),
 array([479.72363], dtype=float32),
 array([484.37314], dtype=float32),
 array([486.55176], dtype=float32),
 array([487.6755], dtype=float32),
 array([486.21002], dtype=float32),
 array([482.4904], dtype=float32),
 array([479.04443], dtype=float32)]